In [42]:
print("Hello")

Hello


In [43]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENROUTER_API_KEY")
print(api_key)

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


In [44]:
from langchain_openai import ChatOpenAI

print("✓ LLM initializing...")

llm = ChatOpenAI(
    model="openai/gpt-4.1-nano",
    openai_api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    temperature=0.5,
    timeout=10
)

print("✓ LLM initialized (OpenAI: gpt-4.1-nano)")

response = llm.invoke("Hello, how are you?")
print(response)


✓ LLM initializing...
✓ LLM initialized (OpenAI: gpt-4.1-nano)
content="Hello! I'm doing well, thank you. How about you? How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'openai/gpt-4.1-nano', 'system_fingerprint': None, 'id': 'gen-1764336899-trrkcIImpGM1bJuRgv4r', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--b87f8efd-f0e0-4129-bcb0-9785c0ca11e9-0' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}


In [45]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

print("✓ Embedding model initialized (HuggingFace: all-MiniLM-L6-v2)")




✓ Embedding model initialized (HuggingFace: all-MiniLM-L6-v2)


In [46]:
from os import path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from pathlib import Path

PDF_PATH = "DSA Handwritten Notes.pdf"

if not Path(PDF_PATH).exists():
    print(f"⚠ Warning: {PDF_PATH} not found.")
    vectorstore = None
    retriever = None
else:
    print(f"📄 Loading PDF: {PDF_PATH}")
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()
    print(f"✓ Loaded {len(docs)} pages")


    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    splits = text_splitter.split_documents(docs)
    print(f"✓ Split into {len(splits)} chunks")

    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
    )

    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 4}
    )

    print(f"✓ Vector store created with {len(splits)} document chunks")
    print("✓ RAG pipeline ready!")

RAG_RETRIEVER = retriever

print("🤖 AI Tutor & Research Agent")
print(RAG_RETRIEVER)

    

📄 Loading PDF: DSA Handwritten Notes.pdf
✓ Loaded 50 pages
✓ Split into 37 chunks
✓ Vector store created with 37 document chunks
✓ RAG pipeline ready!
🤖 AI Tutor & Research Agent
tags=['Chroma', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000020C141B34D0> search_kwargs={'k': 4}


In [47]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

print(search_tool.invoke("What is the capital of France?"))

... what the capital of France is . ... Wait, if I can ’ t look it up right now, but relying on my knowledge, yes, Paris is the capital of France . Besides Paris, what is the capital of France ? ... You use it between your head and your toes, the more it works the thinner it grows. What is the Capital of France ? ... As the capital city of France , the city plays host to the national government of France . However, Paris only became the official capital of France during the reign of Clovis I , in the late 5th and early 6th century. As the capital and largest city in France , it is one of Europe ’ s major centers of finance, diplomacy, commerce, fashion, science and the arts.


that may cause you the error of ddgs if it try to install the ddgs with pip install ddgs 

In [48]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 1. Configure the wrapper to ask for more results
wrapper = DuckDuckGoSearchAPIWrapper(max_results=25) 

# 2. Create the tool using this wrapper
search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)

# 3. Run the search
results = search_tool.invoke("What is the capital of France?")

print(results)


snippet: Paris is the capital and most populous city of France ., title: Paris, France - Intercultural City - Intercultural Cities Programme, link: https://www.coe.int/en/web/interculturalcities/paris, snippet: Paris is the capital city of France , situated on the River Seine, in northern France , at the heart of the Île-de- France region. The Paris "aire urbaine" is one of the most populated areas of its kind in Europe, with a population of roughly ..., title: Paris, France - New World Encyclopedia, link: https://www.newworldencyclopedia.org/entry/Paris,_France, snippet: Exact time now, time zone, time difference, sunrise/sunset time and key facts for Paris, France .Paris is the capital of France ., title: Time in Paris, France now, link: https://time.is/Paris, snippet: Exact time, time zone and time change dates in 2025 for Paris, France .Paris is the capital of France ., title: Current time in Paris, France, link: https://24timezones.com/Paris/time


that is multi result in single line 


In [49]:
import ast
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 1. Setup the search for 25 results
wrapper = DuckDuckGoSearchAPIWrapper(max_results=25)
search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)

# 2. Get the raw string output (the messy part)
raw_output = search_tool.invoke("What is the capital of France?")

# 3. Fix the Output: Convert the string into a real Python list
# The output format typically looks like "[snippet: ..., title: ..., link: ...], ..."
# We can use a custom parser since the default format is sometimes tricky.
# But simpler method first:
try:
    # Try standard list parsing
    results_list = ast.literal_eval(raw_output)
except:
    # Fallback: Manual splitting if ast fails (sometimes LangChain format is weird)
    # This simple split works for the format you pasted
    results_list = raw_output.split(", snippet:") 

print(f"✅ Found {len(results_list)} results:\n")

# 4. Print them cleanly
for i, item in enumerate(results_list, 1):
    # Basic cleanup for display
    clean_item = str(item).replace("snippet: ", "").replace("],", "").strip()
    print(f"Result {i}:")
    print(f"{clean_item[:200]}...") # Print first 200 chars only
    print("-" * 50)


✅ Found 4 results:

Result 1:
With 200,000 inhabitants in 1328, Paris, then already the capital of France , was the most populous city of Europe. By comparison, London in 1300 had 80,000 inhabitants.[29] By the early fourteenth ce...
--------------------------------------------------
Result 2:
Paris is the capital and most populous city of France ., title: Paris, France - Intercultural City - Intercultural Cities Programme, link: https://www.coe.int/en/web/interculturalcities/paris...
--------------------------------------------------
Result 3:
Paris is the capital city of France , situated on the River Seine, in northern France , at the heart of the Île-de- France region. The Paris "aire urbaine" is one of the most populated areas of its ki...
--------------------------------------------------
Result 4:
Exact time now, time zone, time difference, sunrise/sunset time and key facts for Paris, France .Paris is the capital of France ., title: Time in Paris, France now, link: https://time.

In [50]:
# Fix: Use the working DuckDuckGoSearchResults approach
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Create search tool with multiple results
wrapper = DuckDuckGoSearchAPIWrapper(max_results=10)
search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)

print("✓ Web search tool initialized (DuckDuckGo)")
print("✓ Search tool ready for agent use")


✓ Web search tool initialized (DuckDuckGo)
✓ Search tool ready for agent use


In [51]:
# Create RAG Retriever Tool
from langchain_core.tools import Tool

if RAG_RETRIEVER is None:
    print("⚠ Warning: RAG retriever not available. Agent will only use web search.")
    retriever_tool = None
else:
    # Create RAG tool manually (compatible with all LangChain versions)
    def search_document_func(query: str) -> str:
        """Search the uploaded PDF document for relevant information."""
        docs = RAG_RETRIEVER.get_relevant_documents(query)
        if not docs:
            return "No relevant information found in the document."
        
        # Combine all retrieved chunks
        result = "\n\n".join([f"Excerpt {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])
        return result
    
    # Create the tool
    retriever_tool = Tool(
        name="search_document",
        func=search_document_func,
        description="Searches and returns relevant excerpts from the uploaded PDF document. "
        "Use this tool when the user asks questions about the document, specific concepts "
        "mentioned in the document, or requests explanations based on the document content."
    )
    print("✓ RAG retriever tool created")
    print("✓ Document search tool ready")


✓ RAG retriever tool created
✓ Document search tool ready


In [52]:
# Initialize Agent with Memory (Compatible approach - Custom Memory)
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage

# Prepare tools list
tools = [search_tool]
if retriever_tool is not None:
    tools.append(retriever_tool)

print(f"✓ Preparing {len(tools)} tools for agent")

# Simple custom memory class (no external dependencies)
class SimpleMemory:
    """Simple conversation memory that stores messages."""
    def __init__(self):
        self.messages = []
    
    def add_user_message(self, content):
        """Add a user message to memory."""
        self.messages.append(HumanMessage(content=content))
    
    def add_ai_message(self, content):
        """Add an AI message to memory."""
        self.messages.append(AIMessage(content=content))
    
    def get_messages(self, last_n=None):
        """Get messages, optionally limited to last N."""
        if last_n:
            return self.messages[-last_n:]
        return self.messages
    
    def clear(self):
        """Clear all messages."""
        self.messages = []

# Initialize Memory
memory = SimpleMemory()

print("✓ Memory initialized")

# Create a simple agent function that uses tools
def agent_executor(input_dict):
    """Simple agent that routes to appropriate tool."""
    query = input_dict.get("input", "")
    chat_history = input_dict.get("chat_history", [])
    
    # Build context from chat history
    history_text = ""
    if chat_history:
        for msg in chat_history[-4:]:  # Last 4 messages for context
            if hasattr(msg, 'content'):
                role = "Human" if msg.__class__.__name__ == "HumanMessage" else "Assistant"
                history_text += f"{role}: {msg.content}\n"
    
    # Determine which tool to use based on query
    # Simple heuristic: if query mentions "document" or "PDF" or RAG is available, use document search
    use_document = retriever_tool is not None and (
        "document" in query.lower() or 
        "pdf" in query.lower() or 
        "note" in query.lower() or
        "lecture" in query.lower()
    )
    
    # Get response from appropriate tool
    if use_document:
        tool_result = retriever_tool.invoke(query)
        context = f"Document Information:\n{tool_result}\n\n"
    else:
        # Use web search
        tool_result = search_tool.invoke(query)
        context = f"Web Search Results:\n{tool_result}\n\n"
    
    # Create prompt with context
    full_prompt = f"""You are a helpful AI tutor and research assistant.

Previous conversation:
{history_text}

Current query: {query}

Relevant information:
{context}

Based on the information above, provide a clear, helpful answer. If the information doesn't fully answer the question, say so and provide what you can."""
    
    # Get LLM response
    response = llm.invoke(full_prompt)
    return {"output": response.content}

# Create agent as a runnable
agent = RunnableLambda(agent_executor)

print("✓ AI Tutor Agent initialized successfully!")
print("\n🤖 Agent Capabilities:")
print("  • Answer questions from uploaded documents (RAG)")
print("  • Search the web for real-time information")
print("  • Remember conversation context")
print("  • Route queries intelligently")


✓ Preparing 2 tools for agent
✓ Memory initialized
✓ AI Tutor Agent initialized successfully!

🤖 Agent Capabilities:
  • Answer questions from uploaded documents (RAG)
  • Search the web for real-time information
  • Remember conversation context
  • Route queries intelligently


In [53]:
# Interactive Chat Interface
def chat_with_tutor():
    """
    Interactive chat loop with the AI Tutor.
    The agent automatically routes queries to the appropriate tool.
    """
    print("\n" + "="*60)
    print("🤖 AI Tutor & Research Agent")
    print("="*60)
    print("\nI can help you with:")
    print("  • Questions about your uploaded document (RAG)")
    print("  • General knowledge and current events (Web Search)")
    print("  • Explanations and tutoring (Combined knowledge)")
    print("\nType 'quit', 'exit', or 'bye' to end the session.\n")
    
    while True:
        try:
            # Get user input
            user_query = input("\n👤 You: ").strip()
            
            # Check for exit commands
            if user_query.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\n👋 Goodbye! Happy learning!")
                break
            
            if not user_query:
                print("Please enter a question or type 'quit' to exit.")
                continue
            
            # Get agent response
            print("\n🤖 Tutor: ")
            response = agent.invoke({
                "input": user_query,
                "chat_history": memory.get_messages()
            })
            
            # Save to memory
            output_text = response.get("output", "") if isinstance(response, dict) else str(response)
            memory.add_user_message(user_query)
            memory.add_ai_message(output_text)
            
            # Display response
            if isinstance(response, dict) and "output" in response:
                print(response["output"])
            else:
                print(str(response))
                
        except KeyboardInterrupt:
            print("\n\n👋 Session interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            print("Please try rephrasing your question or check your connection.")

print("✓ Chat function ready")
print("💡 Run: chat_with_tutor() to start interactive session")


✓ Chat function ready
💡 Run: chat_with_tutor() to start interactive session


In [54]:
# Quick Test - Single Query
# Uncomment and modify the query to test the agent

# Test 1: Document question (uses RAG if document is loaded)
# if RAG_RETRIEVER is not None:
#     print("📚 Test 1: Document Question")
#     response1 = agent.invoke({
#         "input": "What are the main topics covered in the document?",
#         "chat_history": []
#     })
#     print(f"\nResponse: {response1.get('output', response1) if isinstance(response1, dict) else response1}\n")

# Test 2: Web search question
# print("🌐 Test 2: Web Search Question")
# response2 = agent.invoke({
#     "input": "What is the capital of France?",
#     "chat_history": []
# })
# print(f"\nResponse: {response2.get('output', response2) if isinstance(response2, dict) else response2}\n")

# Test 3: Combined question (uses memory + tools)
# print("💬 Test 3: Follow-up Question (Uses Memory)")
# response3 = agent.invoke({
#     "input": "Can you explain that in simpler terms?",
#     "chat_history": memory.get_messages()
# })
# print(f"\nResponse: {response3.get('output', response3) if isinstance(response3, dict) else response3}")

print("✓ Test queries ready (uncomment to run)")


✓ Test queries ready (uncomment to run)


In [55]:
# Utility Functions
def get_document_summary():
    """
    Get a summary of the uploaded document using RAG.
    """
    if RAG_RETRIEVER is None:
        return "No document loaded."
    
    # Retrieve relevant chunks
    docs = RAG_RETRIEVER.get_relevant_documents("summary of main topics and key concepts")
    
    # Use LLM to generate summary
    context = "\n\n".join([doc.page_content for doc in docs])
    
    prompt = f"""Based on the following document excerpts, provide a concise summary 
    of the main topics and key concepts:
    
    {context}
    
    Summary:"""
    
    response = llm.invoke(prompt)
    return response.content

def clear_memory():
    """
    Clear the conversation memory.
    """
    memory.clear()
    print("✓ Conversation memory cleared")

def get_agent_status():
    """
    Display current agent status and configuration.
    """
    print("🤖 AI Tutor Status:")
    print(f"  • LLM: {llm.model_name}")
    print(f"  • Embeddings: HuggingFace (all-MiniLM-L6-v2)")
    print(f"  • RAG Available: {'Yes' if RAG_RETRIEVER is not None else 'No'}")
    print(f"  • Tools: {len(tools)}")
    print(f"  • Memory: Active")

print("✓ Utility functions ready")
print("  • get_document_summary() - Get document summary")
print("  • clear_memory() - Clear conversation memory")
print("  • get_agent_status() - Show agent status")


✓ Utility functions ready
  • get_document_summary() - Get document summary
  • clear_memory() - Clear conversation memory
  • get_agent_status() - Show agent status


## 🚀 Ready to Use!

**Quick Start:**
1. Run all cells above to initialize the system
2. Use `chat_with_tutor()` to start interactive session
3. Or use `agent.run("your question")` for single queries

**Available Functions:**
- `chat_with_tutor()` - Interactive chat interface
- `get_document_summary()` - Get document summary
- `clear_memory()` - Clear conversation memory
- `get_agent_status()` - Show agent status

**Tips:**
- Ask questions about your document for RAG-based answers
- Ask general questions for web search results
- The agent remembers conversation context automatically
